<a href="https://colab.research.google.com/github/vuchicago/Helloworld/blob/master/SDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
####MOUNT DRIVE
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!pip install tf-nightly-2.0-preview #install tensorflow version #2






In [92]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [93]:
import os
import pandas as pd
import datetime as dt
from datetime import timedelta as td


dir='/content/gdrive/My Drive/' ##original  directory
Ext='US Cement Extraction File' 
Ext_dir=os.path.join(dir,Ext) ##Actuals Directory

adl='SDF_Material_Product_Class'
adl_dir=os.path.join(Ext_dir,adl) ###Materials to Product Directory

sdf_forecast="SDF Outputs"
sdf_dir=os.path.join(dir,sdf_forecast) ###SDF Main Directory
sdf_dir_weekly=os.path.join(sdf_dir,"Weekly") ###Weekly SDF Output Directory
sdf_dir_monthly=os.path.join(sdf_dir,"Monthly") ###Monthly SDF OUtput Directory


tf.__version__



'1.15.0'

In [0]:
os.chdir(adl_dir)
SDF_Product_Class=pd.read_excel("LHUS Material Master Data for Product Class & Form.xlsx",header=0) ###READ IN THE FILE TO MATCH MATERIAL WITH PRODUCT CLASS

In [95]:
###CHOSE THREE FILES FROM DIFFERENT MONTHS THAT CONTAINED ALL THE DATA
os.chdir(Ext_dir)
file4=pd.read_csv('HUS_Cement_Report_20191026.csv')
file3=pd.read_csv('HUS_Cement_Report_20191016.csv')
file2=pd.read_csv('HUS_Cement_Report_20190911.csv')
file1=pd.read_csv('HUS_Cement_Report_201907.csv')

col_keep=['Plant','Material','Product Form','Calendar day','TONS']
file1=file1[col_keep]
file2=file2[col_keep]
file3=file3[col_keep]
file4=file4[col_keep]

file1.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,18,24,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,18,20,24,28,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,18,20,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Plant,Material,Product Form,Calendar day,TONS
0,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-08,1327
1,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-08,1319
2,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-29,1367
3,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-29,1359
4,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-29,1319


In [0]:
###CONVERT CALENDAR DAY TO PYTHON DATE FORMAT
file1["Date"]= file1["Calendar day"].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
file2["Date"]= file2["Calendar day"].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
file3["Date"]= file3["Calendar day"].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
file4["Date"]= file4["Calendar day"].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))

In [0]:
###### OBTAINS ALL THE DATA AND GETS RID OF OVERLAPPING ACTUALS
max_date3=pd.to_datetime(file4["Date"]).min()

file4=file4[file4["Date"]>max_date3]
file3=file3[pd.to_datetime(file3["Date"])<=max_date3]
max_date2=file3["Date"].min()
file2=file2[file2["Date"]<=max_date2]
max_date=file2["Date"].min()
file1=file1[file1["Date"]<=max_date]

In [98]:
##CONCATENATES ALL FILES TOGETHER
files=pd.concat([file1,file2,file3,file4]) 
files.head()

,Plant,Material,Product Form,Calendar day,TONS,Date
0,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-08,1327,2019-07-08
1,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-08,1319,2019-07-08
2,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-29,1367,2019-07-29
3,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-29,1359,2019-07-29
4,2058,"PORTLAND,TYPE I/II,LOW ALKALI LSA",B,2019-07-29,1319,2019-07-29


In [0]:
####MERGES ACTUALS WITH THE PRODUCT CLASS DATA TO OBTAIN THE FIELD "Product_Class"
Master_files=pd.merge(files,SDF_Product_Class,how='left',left_on='Material',right_on='MATERIAL.TXTMD')
Master_files["Plant"]=Master_files["Plant"].astype(str)
Master_files=Master_files.drop(['MATERIAL.MATERIAL','MATERIAL./BIC/ZPACKTYP',"Material",'MATERIAL.TXTMD'],axis=1)
Master_files=Master_files[Master_files["TONS"]!=0]

In [100]:
Master_files=Master_files.rename(columns={'MATERIAL./BIC/ZPRODCLS':"Product_Class",'MATERIAL./BIC/ZPRODTYPE':'Product_Type','MATERIAL./BIC/ZPRODFAM':"Product_Family"})
Master_files.columns

Index(['Plant', 'Product Form', 'Calendar day', 'TONS', 'Date',
       'Product_Class', 'Product_Family', 'Product_Type'],
      dtype='object')

In [0]:
##CREATE THE WEEK COLUMN.  THIS ROUNDS DATE TO THE NEXT WEEKDAY THAT YOU CHOOSE. 6=SUNDAY. WE WILL ROUND TO THE NEAREST FUTURE SUNDAY 
def next_weekday(d, weekday): ###'d' is the date from dataframe.  
    days_ahead = weekday - d.weekday() #'weekday' is the day of the week you want to move ahead (0=Monday, 1=Tuesday, 6=Sunday)
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + td(days_ahead)




In [0]:
####GROUPS THE ACTUAL VOLUMES BY 5 GROUPBY CLASSES.  CREATE THE 'WEEK' DATASET
Master_files["Week"]=Master_files["Date"].apply(lambda x: next_weekday(x,6)) 
Master_files_week=Master_files[["Plant","Product_Class","Product Form","Week","TONS"]]
Master_files_week=Master_files_week.groupby(by=["Plant","Product_Class","Product Form","Week"]).sum().reset_index()
Master_files_week=Master_files_week[Master_files_week['TONS']>0]
Master_files_week["Plant"]=Master_files_week["Plant"]
Master_files_week['Product_Class']=Master_files_week['Product_Class']
Master_files_week['Product Form']=Master_files_week['Product Form']
Master_files_week['Week']=Master_files_week['Week'].map(str).str.strip().str[0:10]
Master_files_week["Week Key"]=Master_files_week['Plant'].map(str)+Master_files_week['Product_Class'].map(str)+Master_files_week['Product Form'].map(str)+Master_files_week['Week'].map(str) ##CREATE WEEK KEY


In [103]:
Master_files_week["Week Key"].head()


0    20581/2B2019-07-07
1    20581/2B2019-07-14
2    20581/2B2019-07-21
3    20581/2B2019-07-28
4    20581/2B2019-08-04
Name: Week Key, dtype: object

In [0]:
# MONTH DATASET
Master_files_month=Master_files
Master_files_month["Month"]=Master_files_month["Date"].apply(lambda x: x.replace(day=1))
Master_files_month=Master_files_month.groupby(by=["Plant","Product_Class","Product Form","Month"]).sum().reset_index()
Master_files_month=Master_files_month[Master_files_month['TONS']>0]
Master_files_month["Plant"]=Master_files_month["Plant"].map(str).str.strip()
Master_files_month['Product_Class']=Master_files_month['Product_Class'].str.strip()
Master_files_month['Product Form']=Master_files_month['Product Form'].str.strip()
Master_files_month['Month']=Master_files_month['Month'].map(str).str.strip().str[0:10]
Master_files_month["Month Key"]=Master_files_month['Plant'].map(str)+Master_files_month['Product_Class'].map(str)+Master_files_month['Product Form'].map(str)+Master_files_month['Month'].map(str) ###CREATE MONTH KEY

In [105]:
Master_files_month

,Plant,Product_Class,Product Form,Month,TONS,Month Key
0,2058,1/2,B,2019-07-01,58717,20581/2B2019-07-01
1,2058,1/2,B,2019-08-01,53903,20581/2B2019-08-01
2,2058,1/2,B,2019-09-01,37894,20581/2B2019-09-01
3,2058,1/2,B,2019-10-01,39025,20581/2B2019-10-01
4,2122,1/2,B,2019-07-01,50299,21221/2B2019-07-01
...,...,...,...,...,...,...
1415,7W02,I..,B,2019-09-01,332,7W02I..B2019-09-01
1416,7W02,I..,B,2019-10-01,421,7W02I..B2019-10-01
1417,7W02,SB.,B,2019-09-01,98,7W02SB.B2019-09-01
1418,7W02,SPE,B,2019-10-01,56,7W02SPEB2019-10-01


In [106]:
Master_files_month["TONS"].sum()

8050680

In [0]:
from pandas import ExcelWriter


In [108]:
import numpy as np
###############################################  WEEKLY #############################################
#### MERGES THE WEEKLY ACTUALS WITH THE WEEKLY FORECASTS.  OUTPUT ALL THE METRICS TO EXCEL FILES
### CREATE A LIST OF ALL THE FILES IN THE FOLDER SO WE CAN LOOP THROUGH THEM
os.chdir(sdf_dir_weekly)
sdf_forecast_files_weekly=os.listdir(sdf_dir_weekly)


writer=pd.ExcelWriter('SDF_weekly.xlsx') ###CREATE THE EXCEL FILE
MAPE=[]
FCA=[]
WAPE=[]
for sdf_files in sdf_forecast_files_weekly:
  actuals=pd.read_csv(sdf_files)
  actuals["Time"]=actuals['forecasted_time'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
  actuals["Plant"]=actuals["Plant"].map(str).str.strip()
  actuals['Product_Class']=actuals['Product_Class'].str.strip()
  actuals['Product_Form']=actuals['Product_Form'].str.strip()
  actuals['Time']=actuals['Time'].map(str).str.strip().str[0:10]
  actuals["Key"]=actuals["Plant"].map(str)+actuals["Product_Class"].map(str)+actuals["Product_Form"].map(str)+actuals["Time"].map(str)

  week=pd.merge(actuals,Master_files_week[["Week Key","TONS"]],how='inner',left_on='Key',right_on="Week Key")
  week=week.rename(columns={"TONS":"Actuals","volume":"Forecast"})
  week["Abs_Dif"]=np.abs(week["Actuals"].subtract(week["Forecast"]))
  week["APE"]=week["Abs_Dif"].divide(week["Actuals"])
  week["one"]=1
  week["APE_max1"]=week[["APE",'one']].min(axis=1)
  week=week.drop(["Time","Key",'one','APE'],axis=1)
  FCA.append(week["Abs_Dif"].sum()/week["Actuals"].sum())
  MAPE.append(sdf_files +": "+ str(week["APE_max1"].mean()) )
  WAPE.append("Wape: "+ str(week["Abs_Dif"].sum()/week["Actuals"].sum()))
  #valuation.append(sdf_files +" : "+ str(tf.keras.metrics.mean_absolute_percentage_error(tf.cast(week["Actuals"],tf.float32),tf.cast(week["Forecast"],tf.float32)).numpy())+'%')
  week.to_excel(writer,sdf_files,index=False) ###ADDS IN TABS FOR THE EXCEL FILE
print("Mean Absolute Percentage Error: ",MAPE) 
print("FCA: ",FCA)
print("WAPE: ", WAPE)


Mean Absolute Percentage Error:  ['20190922_weekly_forecast.csv: 0.4027827188977941', '20190825_weekly_forecast.csv: 0.40566682195186066', '20190725_weekly_forecast.csv: 0.40774680018290277']
FCA:  [0.2464701188223846, 0.2711519113324299, 0.264818432294169]
WAPE:  ['Wape: 0.2464701188223846', 'Wape: 0.2711519113324299', 'Wape: 0.264818432294169']


In [109]:
week[["Week Key","APE_max1"]]

,Week Key,APE_max1
0,20581/2B2019-07-28,0.083946
1,20581/2B2019-08-04,0.217473
2,20581/2B2019-08-11,0.109371
3,20581/2B2019-08-18,0.072263
4,20581/2B2019-08-25,0.047932
...,...,...
3389,5482WP.P2019-09-08,0.732104
3390,54961/2B2019-09-29,0.731253
3391,7W02SB.B2019-09-22,0.118853
3392,7W02SB.B2019-09-29,0.282028


In [111]:
################################################   MONTHLY  #########################################
os.chdir(sdf_dir_monthly)
sdf_forecast_files_monthly=os.listdir(sdf_dir_monthly)
import numpy as np
from matplotlib import pyplot as plt


writer_monthly=pd.ExcelWriter('SDF_monthly.xlsx') ###Create the monthly excel file
MAPE=[]
MAPE_after=[]
FCA=[]
WAPE=[]
actuals=pd.DataFrame()
for sdf_files in sdf_forecast_files_monthly:
  time_cut=int(sdf_files[4:6])+1 ###1 month after only
  ####Read the actual files in 
  actuals=pd.read_csv(sdf_files)
  actuals["Time_temp"]=actuals['forecasted_time'].map(str).apply(lambda x: x[:9]) ###Need this since python was not reading in dates correctly.
  actuals["Time"]=actuals['Time_temp'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%y'))
  actuals["Plant"]=actuals["Plant"].map(str).str.strip() #get rid of leading and trailing blanks
  actuals['Product_Class']=actuals['Product_Class'].replace("2-Jan","1/2").str.strip() ##python would read in 1/2 as 2-Jan.  Probably due to excel formatting issue
  actuals['Product_Form']=actuals['Product_Form'].str.strip()
  actuals['Time']=actuals['Time'].map(str).str.strip().str[0:10]
  actuals["Key"]=actuals["Plant"].map(str)+actuals["Product_Class"].map(str)+actuals["Product_Form"].map(str)+actuals["Time"].map(str)  ##Create a key

  month=pd.merge(actuals,Master_files_month[["Month Key","TONS"]],how='inner',left_on='Key',right_on="Month Key")
  month=month.rename(columns={"TONS":"Actuals","volume":"Forecast"})
  month["month_relevant"]=(month["Time"].map(str).str[5:7]).map(int)
  month["Abs_Dif"]=np.abs(month["Actuals"].subtract(month["Forecast"]))
  month["APE"]=month["Abs_Dif"].divide(month["Actuals"])
  month["one"]=1
  month["APE_max1"]=month[["APE",'one']].min(axis=1)
  month=month.drop(["Time_temp","Key",'one'],axis=1)
  FCA.append(month["Abs_Dif"].sum()/month["Actuals"].sum())
  MAPE.append(sdf_files +": "+ str(month["APE_max1"].mean()) )
  MAPE_after.append(sdf_files+": " + str(month[["APE_max1","month_relevant"]][month["month_relevant"]==time_cut]["APE_max1"].mean()) )
  WAPE.append(month["Abs_Dif"].sum()/month["Actuals"].sum())
  month["Metric"]=""
  month["Metric"][0]=str("MAPE 1 Month After: "+str(MAPE_after[-1]))
  month["Metric"][1]=str("MAPE:"+str(MAPE[-1])) 
  month["Metric"][2]=str("FCA:"+str(FCA[-1]))
  month["Metric"][3]=str("WAPE:"+str(WAPE[-1]))
  month.to_excel(writer_monthly,sdf_files,index=False)


  month_plot=month.set_index("forecasted_time")
  
  
print("Mean Absolute Percentage Error: ",MAPE,"%")
print("Mean Absolute Percentage Error Next Month Only: ",MAPE_after,"%")
print("FCA: ",FCA,"%")
print("MAPE: ",MAPE,"%")
print("WAPE: ",WAPE)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be s

Mean Absolute Percentage Error:  ['20190825_monthly_forecast.csv: 0.46256196210137773', '20190725_monthly_forecast.csv: 0.45469962226100014', '20190922_monthly_forecast.csv: 0.4851494982859487'] %
Mean Absolute Percentage Error Next Month Only:  ['20190825_monthly_forecast.csv: 0.4103271525292033', '20190725_monthly_forecast.csv: 0.4383988453237067', '20190922_monthly_forecast.csv: 0.4851494982859487'] %
FCA:  [0.25899501299079025, 0.24703411606221545, 0.3082269156406397] %
MAPE:  ['20190825_monthly_forecast.csv: 0.46256196210137773', '20190725_monthly_forecast.csv: 0.45469962226100014', '20190922_monthly_forecast.csv: 0.4851494982859487'] %
WAPE:  [0.25899501299079025, 0.24703411606221545, 0.3082269156406397]


In [0]:
time_cut

In [0]:
##WRITE OUT THE MONTHLY FILE
os.chdir(sdf_dir)
##Master_files_month.to_excel("Master_files_month.xlsx")
##Master_files.to_excel("Master_files.xlsx")
writer.save()
writer_monthly.save() 

###################################################################################################################################################################################

###################################################################################################################################################################################

###################################################################################################################################################################################

In [0]:
actuals.head()

EVERYTHING BELOW HERE IS EXPERIMENTALLLLLLLL

EVERYTHING BELOW HERE IS EXPERIMENTAL


EVERYTHING BELOW IS EXPERIMENTAL


In [0]:
from matplotlib import pyplot as plt

month_plot=month.set_index("forecasted_time")
plt.figure(figsize=( 18,18))
month_plot[["Forecast","Actuals"]].groupby(month_plot.index).sum().reset_index().plot(kind='bar',stacked=True,figsize=(28,18))

In [0]:
month_plot

In [0]:
os.chdir(sdf_dir_monthly)
sdf_forecast_files_monthly

In [0]:
check=pd.read_csv("20190725_monthly_forecast.csv")

In [0]:
check[check['forecasted_time']=='09/01/19'].volume.sum()